In [1]:
import pandas as pd
import os
import zipfile
import shutil


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


import warnings
warnings.filterwarnings(action = 'once')#, category = DeprecationWarning)

C:\Users\Jimmy Yang\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Jimmy Yang\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Jimmy Yang\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Jimmy Yang\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: 

In [2]:
#There are more than 43K of training data
#so, we use test dataset as train instead

train_dir = './images/data/test'
valid_dir ='./images/data/valid'

In [3]:
#Count number of image for training
file_count = 0
for root, dirs, files in os.walk(train_dir):
    file_count+=len(files)
print (file_count)

1500


In [4]:
datagen = ImageDataGenerator(
    rotation_range = 60,
    width_shift_range = 0.2,
    height_shift_range=0.2,
    horizontal_flip = True,
    vertical_flip = True,
    zoom_range =0.2
)

In [5]:
#Delete all content under Preview folder
dir = './Preview'
for file in os.listdir(dir):
    path = os.path.join (dir, file)
    os.remove(path)

#get one image and test the datagen above.
img = load_img('./Images/data/one image to test/1.jpg')
x=img_to_array(img)
x= x.reshape((1,) + x.shape)

i=0
for batch in datagen.flow(x,batch_size=1, save_to_dir = 'preview', save_format='jpg', save_prefix='test'):
    i+=1
    if i>10:
        break

In [6]:
from numpy import asarray
data = asarray(img)
input_shape = data.shape

print(input_shape)

(224, 224, 3)


In [7]:
#Show 1 images of each class

In [8]:
#build the model
model = Sequential()

model.add(Conv2D(32,(3,3), padding ='same', activation ='relu', input_shape = input_shape))
model.add(MaxPooling2D())

model.add(Conv2D(32,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3),padding ='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(300))
model.add(Activation('softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


C:\Users\Jimmy Yang\anaconda3\lib\site-packages\tensorflow\python\framework\tensor_util.py:538: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = nparray.tostring()
C:\Users\Jimmy Yang\anaconda3\lib\site-packages\tensorflow\python\ops\nn_ops.py:61: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  elif not isinstance(value, collections.Sized):


In [9]:
#train the model
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(),
             optimizer = 'adam',
             metrics =['accuracy'])

In [10]:
train_datagen = ImageDataGenerator(
    rescale =1./255,
    shear_range =0.2,
    zoom_range= 0.2,
    horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    batch_size =64,
    target_size = (224,224))

validation_generator = test_datagen.flow_from_directory(
        valid_dir,
        batch_size=64,
        target_size=(224, 224))

Found 1500 images belonging to 300 classes.
Found 1500 images belonging to 300 classes.


In [11]:
#fitting the model

dl = model.fit_generator(
    train_generator,
    verbose =1,
    validation_data = validation_generator,
    epochs=150)

model.save_weights('Run_05Oct21.h5')

Epoch 1/150


C:\Users\Jimmy Yang\anaconda3\lib\site-packages\tensorflow\python\framework\tensor_util.py:538: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = nparray.tostring()


24/24 [==============================] - 60s 3s/step - loss: 5.7326 - acc: 0.0020 - val_loss: 5.7038 - val_acc: 0.0040
Epoch 2/150
24/24 [==============================] - 53s 2s/step - loss: 5.7000 - acc: 0.0040 - val_loss: 5.6862 - val_acc: 0.0080
Epoch 3/150
24/24 [==============================] - 62s 3s/step - loss: 5.6724 - acc: 0.0080 - val_loss: 5.6100 - val_acc: 0.0087
Epoch 4/150
24/24 [==============================] - 63s 3s/step - loss: 5.6002 - acc: 0.0113 - val_loss: 5.4979 - val_acc: 0.0167
Epoch 5/150
24/24 [==============================] - 65s 3s/step - loss: 5.4527 - acc: 0.0140 - val_loss: 5.3968 - val_acc: 0.0293
Epoch 6/150
24/24 [==============================] - 61s 3s/step - loss: 5.3403 - acc: 0.0293 - val_loss: 5.2599 - val_acc: 0.0280
Epoch 7/150
24/24 [==============================] - 60s 3s/step - loss: 5.2099 - acc: 0.0293 - val_loss: 5.2507 - val_acc: 0.0340
Epoch 8/150
24/24 [==============================] - 61s 3s/step - loss: 5.0407 - acc: 0.0473 -

24/24 [==============================] - 63s 3s/step - loss: 0.8424 - acc: 0.7533 - val_loss: 5.8914 - val_acc: 0.2447
Epoch 64/150
24/24 [==============================] - 62s 3s/step - loss: 0.9207 - acc: 0.7373 - val_loss: 5.6704 - val_acc: 0.2440
Epoch 65/150
24/24 [==============================] - 62s 3s/step - loss: 0.9129 - acc: 0.7460 - val_loss: 5.8729 - val_acc: 0.2373
Epoch 66/150
24/24 [==============================] - 62s 3s/step - loss: 0.8059 - acc: 0.7600 - val_loss: 5.9323 - val_acc: 0.2473
Epoch 67/150
24/24 [==============================] - 61s 3s/step - loss: 0.7984 - acc: 0.7827 - val_loss: 6.0699 - val_acc: 0.2413
Epoch 68/150
24/24 [==============================] - 62s 3s/step - loss: 0.7967 - acc: 0.7753 - val_loss: 6.3523 - val_acc: 0.2240
Epoch 69/150
24/24 [==============================] - 63s 3s/step - loss: 0.7827 - acc: 0.7607 - val_loss: 6.1016 - val_acc: 0.2427
Epoch 70/150
24/24 [==============================] - 61s 3s/step - loss: 0.7325 - acc: 0

Epoch 125/150
24/24 [==============================] - 62s 3s/step - loss: 0.4314 - acc: 0.8653 - val_loss: 7.1523 - val_acc: 0.2713
Epoch 126/150
24/24 [==============================] - 62s 3s/step - loss: 0.3848 - acc: 0.8860 - val_loss: 7.4613 - val_acc: 0.2653
Epoch 127/150
24/24 [==============================] - 61s 3s/step - loss: 0.4554 - acc: 0.8720 - val_loss: 7.1721 - val_acc: 0.2607
Epoch 128/150
24/24 [==============================] - 60s 3s/step - loss: 0.3890 - acc: 0.8887 - val_loss: 7.5628 - val_acc: 0.2600
Epoch 129/150
24/24 [==============================] - 60s 3s/step - loss: 0.4224 - acc: 0.8720 - val_loss: 7.3427 - val_acc: 0.2593
Epoch 130/150
24/24 [==============================] - 63s 3s/step - loss: 0.4618 - acc: 0.8540 - val_loss: 7.2251 - val_acc: 0.2693
Epoch 131/150
24/24 [==============================] - 62s 3s/step - loss: 0.4109 - acc: 0.8733 - val_loss: 7.5549 - val_acc: 0.2613
Epoch 132/150
24/24 [==============================] - 62s 3s/step - 

AttributeError: 'History' object has no attribute 'summary'

# Read CSV File